# 🐛 5 Game-Changing Tips for Debugging Jupyter Notebooks


We've all been there—staring at a notebook with mysterious errors and no clear path forward. Here's how I've learned to debug more efficiently:

1. Use %debug magic command
When your cell throws an error, run %debug in the next cell. It drops you into an interactive debugger right at the point of failure. Game changer.

2. Print strategically with display()
Instead of cluttering your notebook with print statements, use display() to show DataFrames, images, and objects in a cleaner format. Add print statements at key checkpoints to trace your data flow.

3. Restart kernel + run all cells regularly
Hidden state is your enemy. Variables from previous runs can cause confusing behavior. Make it a habit to restart and run all cells to ensure reproducibility.

4. Break complex cells into smaller ones
If a cell has 50 lines of code, debugging becomes a nightmare. Split logic into smaller, testable chunks. Each cell should do one thing well.

5. Use assert statements liberally
Add assertions to validate your assumptions about data shapes, types, and values throughout your notebook. They catch issues early before they cascade.

  

# 🐛 Jupyter Notebook Debugging Workshop - Google Colab

In [ ]:
# @title Example 1: Post-Mortem Debugging with %debug
# Install ipdb for enhanced debugging
!pip install -q ipdb

import ipdb
import pandas as pd
import numpy as np

print("✓ Setup complete! Ready to debug.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 12.4 MB/s eta 0:00:00
✓ Setup complete! Ready to debug.


In [ ]:
# This code has a bug - run it and it will crash
def calculate_average(numbers):
    total = sum(numbers)
    count = len(numbers)
    return total / count

data = [10, 20, 30, 40, 50]
results = []

for i in range(len(data) + 1):  # Bug: goes one index too far
    # This will fail when i exceeds the list length
    subset = data[i:]
    avg = calculate_average(subset)  # Empty list causes division by zero
    results.append(avg)

print(results)


In [ ]:
# Try these commands in the debugger:​

# p scores - See the empty list
# p len(scores) - Confirms it's 0
# l - List code around the error
# q - Quit the debugger
# c - Continue

%debug

In [ ]:
# @title Example 2: Automatic Debugging with %pdb
# Turn on automatic debugging
%pdb on

def calculate_total(a, b):
    # Bug: will cause division by zero
    result = a / b
    return result

# This will automatically trigger the debugger
total = calculate_total(10, 0)
print(f"Total: {total}")

In [ ]:
# Commands to try:​

# p a
# p b
# p a/b
# q - Quit

In [ ]:
%pdb off

Automatic pdb calling has been turned OFF


In [ ]:
# @title Example 3: Strategic Breakpoints with ipdb.set_trace()
import ipdb
import numpy as np

def train_model_pipeline(data):
    """A complex pipeline with multiple steps"""

    # Step 1: Clean data
    cleaned = data[data > 0]
    print(f"After cleaning: {len(cleaned)} items")

    # Step 2: Transform data
    ipdb.set_trace(context=3)  # PAUSE HERE to inspect

    transformed = np.log(cleaned)

    # Step 3: Calculate statistics
    mean = transformed.mean()
    std = transformed.std()

    ipdb.set_trace()  # PAUSE AGAIN to check results

    return mean, std

# Run the pipeline
raw_data = np.array([1, 2, 3, 4, 5, -1, 0, 10])
mean, std = train_model_pipeline(raw_data)
print(f"Mean: {mean:.2f}, Std: {std:.2f}")

In [ ]:
# @title Example 4: Debugging DataFrames
def analyze_sales(sales_df):
    """Analyze sales data with intentional bugs"""

    # Bug: String in numeric column
    sales_df['revenue'] = sales_df['price'] * sales_df['quantity']

    ipdb.set_trace()  # Pause to inspect data types

    # Bug: This will fail due to data type issues
    total_revenue = sales_df['revenue'].sum()
    avg_revenue = sales_df['revenue'].mean()

    return total_revenue, avg_revenue

# Create sample data with a hidden bug
sales_data = pd.DataFrame({
    'product': ['A', 'B', 'C'],
    'price': [10, '20', 30],  # Bug: '20' is a string!
    'quantity': [5, 3, 8]
})

result = analyze_sales(sales_data)
print(result)

In [ ]:
# @title Example 5: Real-World Machine Learning Bug
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

def prepare_ml_data(X, y):
    """Prepare data for ML model"""

    # Split data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)

    ipdb.set_trace()  # Check shapes before scaling test set

    # Bug: Common mistake - scaling before checking shapes
    X_test_scaled = scaler.transform(X_test)

    return X_train_scaled, X_test_scaled, y_train, y_test

# Generate sample data
X = np.random.rand(100, 5)
y = np.random.randint(0, 2, 100)

X_train, X_test, y_train, y_test = prepare_ml_data(X, y)
print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")


In [1]:
# @title Example 6: Defensive Programming with assert
import pandas as pd
import numpy as np

def preprocess_data(df: pd.DataFrame, feature_columns: list):
    """
    Cleans and prepares data for modeling.
    Uses assert to validate inputs.
    """

    # Assert 1: Check if the input is a pandas DataFrame
    assert isinstance(df, pd.DataFrame), f"Input must be a pandas DataFrame, but got {type(df)}"

    # Assert 2: Check if required columns exist in the DataFrame
    assert all(col in df.columns for col in feature_columns), \
        f"Missing required columns. Expected {feature_columns}, but got {list(df.columns)}"

    # Assert 3: Check for no null values in key columns
    assert df[feature_columns].notna().all().all(), "Null values detected in feature columns."

    # --- Main function logic starts here ---
    print("✓ Assertions passed! Preprocessing data...")

    # Example processing
    df['processed'] = df[feature_columns].sum(axis=1)
    return df

# ---
# Scenario 1: Everything works correctly
# ---
good_data = pd.DataFrame({
    'feature1': [1, 2, 3],
    'feature2': [4, 5, 6],
    'other_col': ['a', 'b', 'c']
})
features = ['feature1', 'feature2']

try:
    processed_df = preprocess_data(good_data, features)
    print("Scenario 1: Success!\n")
    print(processed_df.head())
except AssertionError as e:
    print(f"Scenario 1: Failed with AssertionError: {e}\n")


# ---
# Scenario 2: Fails because of a missing column
# ---
bad_data = pd.DataFrame({
    'feature1': [1, 2, 3],
    # 'feature2' is missing
    'other_col': ['a', 'b', 'c']
})

try:
    preprocess_data(bad_data, features)
except AssertionError as e:
    print(f"\nScenario 2: Failed as expected.\nAssertionError: {e}")


✓ Assertions passed! Preprocessing data...
Scenario 1: Success!

   feature1  feature2 other_col  processed
0         1         4         a          5
1         2         5         b          7
2         3         6         c          9

Scenario 2: Failed as expected.
AssertionError: Missing required columns. Expected ['feature1', 'feature2'], but got ['feature1', 'other_col']


In [ ]:
# @title Additional information
# Quick reference for debugger commands
commands = {
    'Navigation': {
        'n': 'Next line',
        's': 'Step into function',
        'c': 'Continue until breakpoint',
        'r': 'Continue until return',
    },
    'Inspection': {
        'p variable': 'Print variable value',
        'pp variable': 'Pretty print variable',
        'l': 'List code context',
        'w': 'Show stack trace',
        'a': 'Show function arguments',
    },
    'Control': {
        'q': 'Quit debugger',
        'h': 'Show help',
        'h command': 'Help for specific command',
    }
}

for category, cmds in commands.items():
    print(f"\n{category}:")
    for cmd, desc in cmds.items():
        print(f"  {cmd:15} - {desc}")


Navigation:
  n               - Next line
  s               - Step into function
  c               - Continue until breakpoint
  r               - Continue until return

Inspection:
  p variable      - Print variable value
  pp variable     - Pretty print variable
  l               - List code context
  w               - Show stack trace
  a               - Show function arguments

Control:
  q               - Quit debugger
  h               - Show help
  h command       - Help for specific command


In [ ]:
# @title Practice Challenge
def mystery_bug(numbers):
    """Can you find and fix the bug?"""
    result = []
    for i in range(len(numbers)):
        # Hidden bug in this logic
        if numbers[i] > numbers[i+1]:
            result.append(numbers[i])
    return result

# This will crash - use debugging to find out why!
data = [5, 3, 8, 1, 9, 2]
output = mystery_bug(data)
print(output)

#Hint: Use %debug after the crash or add ipdb.set_trace() before the loop!